In [ ]:
# =============================================================================
# Streaming & Async - Real-Time LLM Responses (LangChain 1.0+)
# =============================================================================
"""
=====================================================================
Streaming and Asynchronous Operations
=====================================================================

Why Streaming?
--------------
- Better UX: Users see responses as they're generated
- Lower perceived latency: First token appears quickly
- Memory efficient: Don't buffer entire response

Why Async?
----------
- Handle multiple requests concurrently
- Better performance in web applications
- Non-blocking I/O operations

LangChain Methods:
------------------
Sync:                    Async:
.invoke(input)          .ainvoke(input)
.batch([inputs])        .abatch([inputs])
.stream(input)          .astream(input)
                        .astream_events(input)

Updated for LangChain 1.0+ (2025-2026)
"""

import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

print("✅ Environment configured for streaming examples")

In [ ]:
# =============================================================================
# Basic Streaming with .stream()
# =============================================================================
"""
.stream() Method
----------------
Returns an iterator that yields chunks as they're generated.
Each chunk contains a small piece of the response.

For chat models, chunks are AIMessageChunk objects.
For chains with StrOutputParser, chunks are strings.
"""

from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize model
model = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=0.7)

print("=" * 60)
print("Basic Streaming Example")
print("=" * 60)

print("\n📤 Streaming response (token by token):\n")

# Stream from model directly
for chunk in model.stream([HumanMessage(content="Write a haiku about Python programming")]):
    # chunk.content contains the text piece
    print(chunk.content, end="", flush=True)

print("\n\n✅ Streaming complete!")

In [ ]:
# =============================================================================
# Streaming with LCEL Chains
# =============================================================================
"""
Streaming Through Chains
------------------------
When you stream through a chain, the streaming happens
at the model step and propagates through parsers.

Chain: prompt → model (streams) → parser (passes chunks)
"""

print("=" * 60)
print("Streaming Through LCEL Chain")
print("=" * 60)

# Create a chain
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a creative storyteller."),
    ("human", "Write a very short story about {topic} in 3 sentences.")
])

chain = prompt | model | StrOutputParser()

print("\n📖 Streaming story about 'a robot learning to cook':\n")

# Stream through the entire chain
for chunk in chain.stream({"topic": "a robot learning to cook"}):
    print(chunk, end="", flush=True)

print("\n\n✅ Chain streaming complete!")

In [ ]:
# =============================================================================
# Async Operations with asyncio
# =============================================================================
"""
Async Methods (a-prefix)
------------------------
All runnables support async versions:
- await chain.ainvoke(input)
- await chain.abatch([inputs])
- async for chunk in chain.astream(input)

Benefits:
- Non-blocking: Other tasks can run while waiting for LLM
- Concurrent requests: Handle multiple users simultaneously
- Better performance in web frameworks (FastAPI, etc.)
"""

import asyncio

print("=" * 60)
print("Async Operations Example")
print("=" * 60)

async def async_invoke_example():
    """Example of async invoke."""
    print("\n1️⃣ Async invoke (single request):")
    
    result = await chain.ainvoke({"topic": "a time-traveling cat"})
    print(f"   {result[:100]}...")
    return result

async def async_batch_example():
    """Example of async batch - process multiple inputs concurrently."""
    print("\n2️⃣ Async batch (multiple requests in parallel):")
    
    topics = [
        {"topic": "a magic coffee machine"},
        {"topic": "a talking houseplant"},
        {"topic": "a detective dog"},
    ]
    
    # All three requests run concurrently!
    results = await chain.abatch(topics)
    
    for i, (topic, result) in enumerate(zip(topics, results)):
        print(f"   Story {i+1} ({topic['topic']}): {result[:50]}...")
    
    return results

async def async_stream_example():
    """Example of async streaming."""
    print("\n3️⃣ Async streaming:")
    
    print("   ", end="")
    async for chunk in chain.astream({"topic": "a singing laptop"}):
        print(chunk, end="", flush=True)
    print()

# Run all async examples
async def main():
    await async_invoke_example()
    await async_batch_example()
    await async_stream_example()

# In Jupyter, use await directly
await main()

In [ ]:
# =============================================================================
# Concurrent Async Requests
# =============================================================================
"""
Running Multiple LLM Calls Concurrently
---------------------------------------
Use asyncio.gather() to run multiple async calls at once.
This is faster than sequential calls.

Timing comparison:
- Sequential: 3 calls × 2s each = 6s total
- Concurrent: 3 calls running together = ~2s total
"""

import time

print("=" * 60)
print("Concurrent vs Sequential Comparison")
print("=" * 60)

topics = [
    {"topic": "artificial intelligence"},
    {"topic": "quantum computing"},
    {"topic": "biotechnology"},
]

# Sequential approach
async def sequential_calls():
    results = []
    start = time.time()
    
    for topic in topics:
        result = await chain.ainvoke(topic)
        results.append(result)
    
    elapsed = time.time() - start
    return results, elapsed

# Concurrent approach using gather
async def concurrent_calls():
    start = time.time()
    
    # Create all tasks
    tasks = [chain.ainvoke(topic) for topic in topics]
    
    # Run all concurrently
    results = await asyncio.gather(*tasks)
    
    elapsed = time.time() - start
    return results, elapsed

# Compare both approaches
async def compare():
    print("\n⏱️ Sequential calls (one after another):")
    seq_results, seq_time = await sequential_calls()
    print(f"   Time: {seq_time:.2f}s")
    
    print("\n⚡ Concurrent calls (all at once):")
    con_results, con_time = await concurrent_calls()
    print(f"   Time: {con_time:.2f}s")
    
    print(f"\n📊 Speedup: {seq_time/con_time:.1f}x faster with concurrency!")

await compare()

In [ ]:
# =============================================================================
# Streaming Events with astream_events()
# =============================================================================
"""
.astream_events() Method
------------------------
Provides detailed streaming events from ALL steps in a chain.
Useful for:
- Debugging complex chains
- Progress tracking
- Detailed logging

Event Types:
- on_chain_start, on_chain_end
- on_llm_start, on_llm_stream, on_llm_end
- on_tool_start, on_tool_end
"""

print("=" * 60)
print("Streaming Events Example")
print("=" * 60)

async def stream_events_example():
    print("\n📊 Streaming events from chain:\n")
    
    async for event in chain.astream_events(
        {"topic": "a friendly alien"},
        version="v2"  # Use v2 for latest event format
    ):
        kind = event["event"]
        
        if kind == "on_chain_start":
            print(f"🔗 Chain started: {event['name']}")
        
        elif kind == "on_chat_model_stream":
            # This is where tokens are streamed
            content = event["data"]["chunk"].content
            if content:
                print(content, end="", flush=True)
        
        elif kind == "on_chain_end":
            print(f"\n\n✅ Chain ended: {event['name']}")

await stream_events_example()

In [ ]:
# =============================================================================
# Callbacks for Monitoring
# =============================================================================
"""
Callback Handlers
-----------------
Callbacks let you hook into the execution lifecycle.
Built-in handlers:
- StdOutCallbackHandler - Print to console
- StreamingStdOutCallbackHandler - Stream tokens to console

You can also create custom handlers.
"""

from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.outputs import LLMResult
from typing import Any, Dict, List

print("=" * 60)
print("Custom Callback Handler Example")
print("=" * 60)

class TokenCounterCallback(BaseCallbackHandler):
    """Custom callback that counts tokens and measures time."""
    
    def __init__(self):
        self.token_count = 0
        self.start_time = None
    
    def on_llm_start(self, serialized: Dict[str, Any], prompts: List[str], **kwargs):
        self.start_time = time.time()
        print(f"\n🚀 LLM started")
    
    def on_llm_new_token(self, token: str, **kwargs):
        self.token_count += 1
        print(token, end="", flush=True)
    
    def on_llm_end(self, response: LLMResult, **kwargs):
        elapsed = time.time() - self.start_time
        print(f"\n\n📊 Stats: {self.token_count} tokens in {elapsed:.2f}s")
        print(f"   Speed: {self.token_count/elapsed:.1f} tokens/second")

# Use the callback
callback = TokenCounterCallback()

# Invoke with callback
result = model.invoke(
    [HumanMessage(content="Explain what streaming means in 2 sentences.")],
    config={"callbacks": [callback]}
)

In [ ]:
# =============================================================================
# FastAPI Integration Pattern
# =============================================================================
"""
Streaming with FastAPI
----------------------
Real-world pattern for streaming LLM responses in web apps.
Uses Server-Sent Events (SSE) for real-time streaming.
"""

print("=" * 60)
print("FastAPI Streaming Pattern")
print("=" * 60)

print("""
# FastAPI streaming endpoint example:

from fastapi import FastAPI
from fastapi.responses import StreamingResponse
from langchain.chat_models import init_chat_model

app = FastAPI()
model = init_chat_model("gpt-4o-mini", model_provider="openai")

@app.post("/stream")
async def stream_response(query: str):
    
    async def generate():
        async for chunk in model.astream([HumanMessage(content=query)]):
            if chunk.content:
                yield f"data: {chunk.content}\\n\\n"
        yield "data: [DONE]\\n\\n"
    
    return StreamingResponse(
        generate(),
        media_type="text/event-stream"
    )

# Frontend JavaScript:
const eventSource = new EventSource('/stream?query=Hello');
eventSource.onmessage = (event) => {
    if (event.data === '[DONE]') {
        eventSource.close();
    } else {
        document.getElementById('output').innerHTML += event.data;
    }
};
""")

In [ ]:
# =============================================================================
# Summary: Streaming & Async in LangChain 1.0+
# =============================================================================
"""
=====================================================================
KEY TAKEAWAYS - Streaming & Async
=====================================================================

1. STREAMING METHODS:
   ------------------
   # Sync streaming
   for chunk in chain.stream(input):
       print(chunk, end="")
   
   # Async streaming
   async for chunk in chain.astream(input):
       print(chunk, end="")

2. ASYNC METHODS:
   ---------------
   result = await chain.ainvoke(input)    # Single async
   results = await chain.abatch([inputs]) # Batch async

3. CONCURRENT EXECUTION:
   ----------------------
   # Run multiple calls in parallel
   tasks = [chain.ainvoke(inp) for inp in inputs]
   results = await asyncio.gather(*tasks)

4. STREAMING EVENTS:
   ------------------
   async for event in chain.astream_events(input, version="v2"):
       if event["event"] == "on_chat_model_stream":
           print(event["data"]["chunk"].content)

5. CALLBACKS:
   -----------
   from langchain_core.callbacks import BaseCallbackHandler
   
   class MyCallback(BaseCallbackHandler):
       def on_llm_new_token(self, token, **kwargs):
           print(token, end="")
   
   model.invoke(messages, config={"callbacks": [MyCallback()]})

6. BEST PRACTICES:
   ----------------
   - Use streaming for better UX
   - Use async in web applications
   - Use concurrent calls for multiple independent requests
   - Use callbacks for monitoring/logging
   - Handle timeouts and errors gracefully

Common Imports:
---------------
import asyncio
from langchain_core.callbacks import BaseCallbackHandler
from langchain.chat_models import init_chat_model

=====================================================================
"""

print("=" * 60)
print("Streaming & Async Module Complete!")
print("=" * 60)
print("""
Key Patterns:
-------------
1. .stream() / .astream() for real-time output
2. .ainvoke() / .abatch() for async operations
3. asyncio.gather() for concurrent requests
4. Callbacks for monitoring and logging
5. FastAPI + SSE for web streaming

Next: 9-RAG.ipynb - Retrieval Augmented Generation
""")